# 라이브러리 불러오기

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train= pd.read_csv("train_서울대공원_혼잡도예측_프로젝트.csv")
test=pd.read_csv("test_서울대공원_혼잡도예측_프로젝트.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# train+test 데이터 통합

같이 전처리 후 분할 예정

ntrain/ntest 저장을 위해 행 삭제하는 전처리 미리 진행

In [ ]:

#임시휴원 있는 행 삭제
spac=train[train['입장객 수']=='임시휴원'].index
train.drop(spac,inplace=True)

#test data의 y값 '정보없음' 행 삭제
test=test.drop([249])

all_data = pd.concat((train, test)).reset_index(drop=True)

ntrain = train.shape[0]
ntest = test.shape[0]


In [ ]:
#날짜 분리하여 새로운 변수 만들기
all_data[['년','월','일']] = all_data['날짜'].str.split('.', expand=True)
all_data.drop(['날짜'], axis=1, inplace=True)


#임시휴원 있는 행 삭제
spac=all_data[all_data['입장객 수']=='임시휴원'].index
all_data.drop(spac,inplace=True)
all_data.isnull().sum()

#미세먼지 농도 - -> NAN값 처리
all_data['미세먼지 농도']=all_data['미세먼지 농도'].replace('-', np.NaN)

#공휴일 여부 -> 0,1로 변환
all_data['공휴일여부']=all_data['공휴일여부'].replace(np.NaN, 0)
all_data['공휴일여부']=all_data['공휴일여부'].replace('공휴일', 1)

##########################################################
#'강우여부' 파생변수 생성, 누적강수량 제거
all_data['강우여부'] = np.where(all_data['누적 강수량'] == 0, 0, 1)
all_data.drop('누적 강수량', axis = 1, inplace = True)

#'쉬는날여부' 제거, '주말여부' 파생변수 생성
all_data.drop('쉬는날여부', axis = 1, inplace = True)
all_data['주말여부'] = np.where((all_data['요일'] == '토') | (all_data['요일'] == '일'), 1, 0)

#'평균기온', '최저기온','최고기온'을 '기온'으로 통일
all_data['기온'] = np.where(all_data['계절'] == '겨울', all_data['최저기온'], np.where(all_data['계절'] == '여름', all_data['최고기온'], all_data['평균기온']))
all_data.drop(['평균기온', '최저기온', '최고기온'], axis = 1, inplace = True)

############################################################


#데이터 타입 바꾸기
all_data = all_data.astype({'년':'int'})
all_data = all_data.astype({'월':'int'})
all_data = all_data.astype({'일':'int'})
all_data = all_data.astype({'미세먼지 농도':'float64'})
all_data = all_data.astype({'입장객 수':'float64'})         

#변수 정렬
all_data=all_data[['년', '월', '일','요일', '계절', '공휴일여부','주말여부', '방학여부', '강우여부', '평균 습도', '입장객 수', '대공원역 하차자수',
       '미세먼지 농도', '기온']]

#변수 영어로 바꾸기
all_data.columns = ['Year','Month','Date','Day','Season','Pub_holiday', 'Weekend','Vacation','Precipitation_accum','Humidity_avg','Attendance','Get_off_subway','Fine_dust_concentration',
                 'Temperature']

#변수 영어로 바꾸기2
all_data = all_data.replace({'Day' : '월'},'Mon')
all_data = all_data.replace({'Day' : '화'},'Tue')
all_data = all_data.replace({'Day' : '수'},'Wed')
all_data = all_data.replace({'Day' : '목'},'Thu')
all_data = all_data.replace({'Day' : '금'},'Fri')
all_data = all_data.replace({'Day' : '토'},'Sat')
all_data = all_data.replace({'Day' : '일'},'Sun')

all_data = all_data.replace({'Season' : '봄'},'Spring')
all_data = all_data.replace({'Season' : '여름'},'Summer')
all_data = all_data.replace({'Season' : '가을'},'fall')
all_data = all_data.replace({'Season' : '겨울'},'Winter')   

# 전처리

1. 미세먼지 결측값-> 하차자수와 미세먼지 변수의 단일선형회귀로 얻은 예측값으로 대체

(이유: factor analysis 결과, 미세먼지와 하차자수 사이에 밀접한 관련이 있음을 확인)

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()

X= all_data.dropna(axis=0)['Get_off_subway']
y= all_data.dropna(axis=0)['Fine_dust_concentration']


X=pd.DataFrame(X,index=X.index)
y=pd.DataFrame(y,index=y.index)

lin_reg_model = lin_reg.fit(X, y)
y_pred = lin_reg_model.predict(all_data.loc[:, ['Get_off_subway']])


all_data['Fine_dust_concentration'] = np.where(all_data['Fine_dust_concentration'].isnull(), 
                              pd.Series(y_pred.flatten()), 
                              all_data['Fine_dust_concentration'])

2. 원핫인코딩

In [ ]:
all_data_final=pd.get_dummies(all_data)

3. 분석에 필요없는 변수 지우기

In [ ]:
all_data_final.drop(columns=["Year", "Month","Date"],inplace=True)

4. y값 로그변환

In [ ]:
all_data_final['Attendance']=np.log1p(all_data_final['Attendance'])

## train+test 다시 split

In [ ]:
train = all_data_final[:ntrain]
test = all_data_final[ntrain:]

test.reset_index(drop= True, inplace = True)

In [ ]:
test.isnull().sum()

Pub_holiday                0
Weekend                    0
Vacation                   0
Precipitation_accum        0
Humidity_avg               0
Attendance                 0
Get_off_subway             0
Fine_dust_concentration    0
Temperature                0
Day_Fri                    0
Day_Mon                    0
Day_Sat                    0
Day_Sun                    0
Day_Thu                    0
Day_Tue                    0
Day_Wed                    0
Season_Spring              0
Season_Summer              0
Season_Winter              0
Season_fall                0
dtype: int64

# 전처리2

표준화

In [ ]:
#train data 처리
train_num=train[['Humidity_avg','Get_off_subway','Fine_dust_concentration','Temperature']]
train_obj=train.drop(['Humidity_avg','Get_off_subway','Attendance','Fine_dust_concentration','Temperature'],axis=1)

#표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train_num)
train_std = pd.DataFrame(scaler.transform(train_num), columns=train_num.columns, index=train_num.index) 

#최종 train_x와 train_y
train_final_x=pd.concat([train_std,train_obj],axis=1)
train_final_y=train['Attendance']

###########################################################################################################

#test data 처리
test_num=test[['Humidity_avg','Get_off_subway','Fine_dust_concentration','Temperature']]
test_obj=test.drop(['Humidity_avg','Get_off_subway','Attendance','Fine_dust_concentration','Temperature'],axis=1)

#표준화(test data에는 transform만 적용)
test_std = pd.DataFrame(scaler.transform(test_num), columns=test_num.columns, index=test_num.index) 

#최종 test_x와 test_y
test_final_x=pd.concat([test_std,test_obj],axis=1)
test_final_y=test['Attendance']

# 모델링

In [ ]:
!pip install optuna
!pip install catboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import catboost as cb
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import PredefinedSplit
from sklearn import ensemble
import numpy as np
from sklearn.model_selection import KFold


def rmse_cv(model):
    rmse = -cross_val_score(model, train_final_x,train_final_y, scoring="neg_root_mean_squared_error", cv=5).mean()
    return rmse
def evaluation(y, predictions):
    rmse = np.sqrt(mean_squared_error(y, predictions))
    return rmse    

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 8.0 MB/s 
     |████████████████████████████████| 81 kB 4.3 MB/s 
     |████████████████████████████████| 209 kB 25.9 MB/s 
     |████████████████████████████████| 78 kB 2.9 MB/s 
     |████████████████████████████████| 146 kB 25.5 MB/s 
     |████████████████████████████████| 112 kB 23.8 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=99139d69061df2e2f8987179ba86e310465b1f2a82d53e89423b09c492698d5b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 1.1 MB/s 


In [ ]:
OPTUNA_EARLY_STOPING = 30

class EarlyStoppingExceeded(optuna.exceptions.OptunaError):
    early_stop = OPTUNA_EARLY_STOPING
    early_stop_count = 0
    best_score = None

def early_stopping_opt(study, trial):
    if EarlyStoppingExceeded.best_score == None:
      EarlyStoppingExceeded.best_score = study.best_value

    if study.best_value < EarlyStoppingExceeded.best_score:
        EarlyStoppingExceeded.best_score = study.best_value
        EarlyStoppingExceeded.early_stop_count = 0
    else:
      if EarlyStoppingExceeded.early_stop_count > EarlyStoppingExceeded.early_stop:
            EarlyStoppingExceeded.early_stop_count = 0
            best_score = None
            raise EarlyStoppingExceeded()
      else:
            EarlyStoppingExceeded.early_stop_count=EarlyStoppingExceeded.early_stop_count+1
    #print(f'EarlyStop counter: {EarlyStoppingExceeded.early_stop_count}, Best score: {study.best_value} and {EarlyStoppingExceeded.best_score}')
    return

### GradientBoosting regressor

In [ ]:
def objective(trial):
    max_depth = trial.suggest_int('max_depth', 1, 1000)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-8, 1e-2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 1000)
    n_estimators =  trial.suggest_int('n_estimators', 30, 1000)
    loss = trial.suggest_categorical('loss', ['ls'])

    
    regr = ensemble.GradientBoostingRegressor(max_depth = max_depth, learning_rate = learning_rate,
                                 min_samples_leaf = min_samples_leaf,n_estimators = n_estimators,loss=loss)
    score = cross_val_score(regr,train_final_x,train_final_y, cv=KFold(n_splits=10,shuffle=True,random_state=42), scoring="neg_mean_squared_error")
    neg_mean_squared_error_mean = score.mean()

    return neg_mean_squared_error_mean

In [ ]:
#Execute optuna and set hyperparameters
study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=500, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')

[I 2022-07-05 18:53:27,044] A new study created in memory with name: no-name-35310baa-e00c-4eb2-a6d5-7090a1ba2ab2
[I 2022-07-05 18:53:32,586] Trial 0 finished with value: -1.4004975699900675 and parameters: {'max_depth': 3, 'learning_rate': 3.078645388269381e-06, 'min_samples_leaf': 432, 'n_estimators': 807, 'loss': 'ls'}. Best is trial 0 with value: -1.4004975699900675.
[I 2022-07-05 18:53:33,706] Trial 1 finished with value: -1.4041052547383992 and parameters: {'max_depth': 398, 'learning_rate': 0.00035480632074027623, 'min_samples_leaf': 958, 'n_estimators': 458, 'loss': 'ls'}. Best is trial 0 with value: -1.4004975699900675.
[I 2022-07-05 18:53:33,910] Trial 2 finished with value: -1.4041052547383992 and parameters: {'max_depth': 916, 'learning_rate': 4.522211185161404e-05, 'min_samples_leaf': 999, 'n_estimators': 72, 'loss': 'ls'}. Best is trial 0 with value: -1.4004975699900675.
[I 2022-07-05 18:53:35,830] Trial 3 finished with value: -1.4041052547383992 and parameters: {'max_dep

EarlyStopping Exceeded: No new best scores on iters 30


In [ ]:
Best_trial = study.best_trial.params
optimised_GB = ensemble.GradientBoostingRegressor(**Best_trial)
optimised_GB.fit(train_final_x,train_final_y)
mse = mean_squared_error(train_final_y, optimised_GB.predict(train_final_x))

print('최적의 하이퍼파라미터',study.best_trial.params)
print('최적 모델의 cv score', np.sqrt(mse))
print('최적 모델',study.best_trial )

models=pd.DataFrame(columns=["Model","최적 모델의 cv_RMSE"])
new_row = {"Model": "GradientBoost","최적 모델의 cv_RMSE": np.sqrt(mse)}
models = models.append(new_row,ignore_index=True)
models

최적의 하이퍼파라미터 {'max_depth': 11, 'learning_rate': 0.009794933198227902, 'min_samples_leaf': 23, 'n_estimators': 976, 'loss': 'ls'}
최적 모델의 cv score 0.25202669436716857
최적 모델 FrozenTrial(number=11, values=[-0.2081977359935853], datetime_start=datetime.datetime(2022, 7, 5, 18, 54, 19, 205281), datetime_complete=datetime.datetime(2022, 7, 5, 18, 54, 48, 914567), params={'max_depth': 11, 'learning_rate': 0.009794933198227902, 'min_samples_leaf': 23, 'n_estimators': 976, 'loss': 'ls'}, distributions={'max_depth': IntUniformDistribution(high=1000, low=1, step=1), 'learning_rate': LogUniformDistribution(high=0.01, low=1e-08), 'min_samples_leaf': IntUniformDistribution(high=1000, low=1, step=1), 'n_estimators': IntUniformDistribution(high=1000, low=30, step=1), 'loss': CategoricalDistribution(choices=('ls',))}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=11, state=TrialState.COMPLETE, value=None)


,Model,최적 모델의 cv_RMSE
0,GradientBoost,0.252027


### CatBoost

In [ ]:
def objective(trial):

    cbrm_param = {
        'iterations':trial.suggest_int("iterations", 4000, 25000),
        'od_wait':trial.suggest_int('od_wait', 500, 2300),
        'learning_rate' : trial.suggest_uniform('learning_rate',0.01, 1),
        'reg_lambda': trial.suggest_uniform('reg_lambda',1e-5,100),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'random_strength': trial.suggest_uniform('random_strength',10,50),
        'depth': trial.suggest_int('depth',1, 15),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,30),
        'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00),
        'colsample_bylevel':trial.suggest_float('colsample_bylevel', 0.4, 1.0),
    }

    # Generate model
    regr = cb.CatBoostRegressor(**cbrm_param)
    score = cross_val_score(regr,train_final_x,train_final_y, cv=KFold(n_splits=10,shuffle=True,random_state=42), scoring="neg_mean_squared_error")
    neg_mean_squared_error_mean = score.mean()

    return neg_mean_squared_error_mean

In [ ]:
#Execute optuna and set hyperparameters
study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=5, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')

[I 2022-07-05 19:58:36,645] A new study created in memory with name: no-name-84037b3f-1835-4f23-98a1-b7434d2029f1


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
14994:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
14995:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
14996:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
14997:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
14998:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
14999:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15000:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15001:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15002:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15003:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15004:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15005:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15006:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15007:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15008:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15009:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15010:	learn: 0.0000000	total: 1m 19s	remaining: 26.3s
15011:	learn: 0.0000000	total

[I 2022-07-05 20:18:00,700] Trial 0 finished with value: -0.230803556296362 and parameters: {'iterations': 19994, 'od_wait': 1407, 'learning_rate': 0.48940753777266194, 'reg_lambda': 48.37851174983393, 'subsample': 0.21199339043789556, 'random_strength': 41.10508324200043, 'depth': 9, 'min_data_in_leaf': 16, 'leaf_estimation_iterations': 2, 'bagging_temperature': 25.530357632825687, 'colsample_bylevel': 0.8926304701143835}. Best is trial 0 with value: -0.230803556296362.


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
7600:	learn: 0.0000790	total: 6.35s	remaining: 4.24s
7601:	learn: 0.0000789	total: 6.36s	remaining: 4.24s
7602:	learn: 0.0000789	total: 6.36s	remaining: 4.24s
7603:	learn: 0.0000789	total: 6.36s	remaining: 4.24s
7604:	learn: 0.0000788	total: 6.36s	remaining: 4.24s
7605:	learn: 0.0000785	total: 6.36s	remaining: 4.24s
7606:	learn: 0.0000785	total: 6.36s	remaining: 4.23s
7607:	learn: 0.0000784	total: 6.36s	remaining: 4.23s
7608:	learn: 0.0000782	total: 6.36s	remaining: 4.23s
7609:	learn: 0.0000781	total: 6.36s	remaining: 4.23s
7610:	learn: 0.0000781	total: 6.36s	remaining: 4.23s
7611:	learn: 0.0000781	total: 6.36s	remaining: 4.23s
7612:	learn: 0.0000781	total: 6.37s	remaining: 4.23s
7613:	learn: 0.0000780	total: 6.37s	remaining: 4.23s
7614:	learn: 0.0000779	total: 6.37s	remaining: 4.23s
7615:	learn: 0.0000779	total: 6.37s	remaining: 4.23s
7616:	learn: 0.0000778	total: 6.37s	remaining: 4.23s
7617:	learn: 0.0000777	total: 6.37s	remaining: 4.22s
7618:	lear

[I 2022-07-05 20:20:04,648] Trial 1 finished with value: -0.2528243134548814 and parameters: {'iterations': 12672, 'od_wait': 2208, 'learning_rate': 0.6224520721615043, 'reg_lambda': 32.75135571427973, 'subsample': 0.4759264733861116, 'random_strength': 19.689114118456423, 'depth': 3, 'min_data_in_leaf': 10, 'leaf_estimation_iterations': 11, 'bagging_temperature': 0.018276370335807417, 'colsample_bylevel': 0.794175493336835}. Best is trial 0 with value: -0.230803556296362.


...

[I 2022-07-05 20:25:15,329] Trial 2 finished with value: -0.2452267749927875 and parameters: {'iterations': 17394, 'od_wait': 1601, 'learning_rate': 0.49500616354811167, 'reg_lambda': 63.069573993503894, 'subsample': 0.5277099922800979, 'random_strength': 43.56871549543186, 'depth': 7, 'min_data_in_leaf': 8, 'leaf_estimation_iterations': 5, 'bagging_temperature': 0.36931896266571296, 'colsample_bylevel': 0.5267882460551497}. Best is trial 0 with value: -0.230803556296362.


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5637:	learn: 0.0079432	total: 9.84s	remaining: 8.72s
5638:	learn: 0.0079397	total: 9.84s	remaining: 8.72s
5639:	learn: 0.0079370	total: 9.84s	remaining: 8.72s
5640:	learn: 0.0079328	total: 9.84s	remaining: 8.72s
5641:	learn: 0.0079296	total: 9.85s	remaining: 8.72s
5642:	learn: 0.0079273	total: 9.85s	remaining: 8.71s
5643:	learn: 0.0079229	total: 9.85s	remaining: 8.71s
5644:	learn: 0.0079185	total: 9.85s	remaining: 8.71s
5645:	learn: 0.0079123	total: 9.85s	remaining: 8.71s
5646:	learn: 0.0079071	total: 9.85s	remaining: 8.71s
5647:	learn: 0.0079011	total: 9.85s	remaining: 8.7s
5648:	learn: 0.0078990	total: 9.86s	remaining: 8.7s
5649:	learn: 0.0078957	total: 9.86s	remaining: 8.7s
5650:	learn: 0.0078904	total: 9.86s	remaining: 8.7s
5651:	learn: 0.0078877	total: 9.86s	remaining: 8.7s
5652:	learn: 0.0078830	total: 9.86s	remaining: 8.69s
5653:	learn: 0.0078753	total: 9.86s	remaining: 8.69s
5654:	learn: 0.0078717	total: 9.86s	remaining: 8.69s
5655:	learn: 0.

[I 2022-07-05 20:28:19,878] Trial 3 finished with value: -0.2083328624582681 and parameters: {'iterations': 10637, 'od_wait': 1789, 'learning_rate': 0.09404895730815328, 'reg_lambda': 51.08340432646926, 'subsample': 0.4808467400433857, 'random_strength': 21.485995983978867, 'depth': 7, 'min_data_in_leaf': 1, 'leaf_estimation_iterations': 2, 'bagging_temperature': 0.7489081189661576, 'colsample_bylevel': 0.4720188004073166}. Best is trial 3 with value: -0.2083328624582681.


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5082:	learn: 0.0000000	total: 44.6s	remaining: 43.9s
5083:	learn: 0.0000000	total: 44.6s	remaining: 43.9s
5084:	learn: 0.0000000	total: 44.6s	remaining: 43.9s
5085:	learn: 0.0000000	total: 44.6s	remaining: 43.9s
5086:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5087:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5088:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5089:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5090:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5091:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5092:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5093:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5094:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5095:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5096:	learn: 0.0000000	total: 44.7s	remaining: 43.8s
5097:	learn: 0.0000000	total: 44.7s	remaining: 43.7s
5098:	learn: 0.0000000	total: 44.8s	remaining: 43.7s
5099:	learn: 0.0000000	total: 44.8s	remaining: 43.7s
5100:	lear

[I 2022-07-05 20:44:36,743] Trial 4 finished with value: -0.22340571536287177 and parameters: {'iterations': 10082, 'od_wait': 1453, 'learning_rate': 0.12247888409905473, 'reg_lambda': 34.65751061850178, 'subsample': 0.3491713465019176, 'random_strength': 43.12735077788202, 'depth': 10, 'min_data_in_leaf': 28, 'leaf_estimation_iterations': 14, 'bagging_temperature': 0.1837477833872965, 'colsample_bylevel': 0.8412305122990895}. Best is trial 3 with value: -0.2083328624582681.


In [ ]:
Best_trial = study.best_trial.params
optimised_cb = cb.CatBoostRegressor(**Best_trial)
optimised_cb.fit(train_final_x,train_final_y)
mse = mean_squared_error(train_final_y, optimised_cb.predict(train_final_x))

print('최적의 하이퍼파라미터',study.best_trial.params)
print('최적 모델의 cv score', np.sqrt(mse))
print('최적 모델',study.best_trial )

new_row = {"Model": "CatBoost","최적 모델의 cv_RMSE": np.sqrt(mse)}
models = models.append(new_row,ignore_index=True)
models

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5640:	learn: 0.0099211	total: 10.9s	remaining: 9.65s
5641:	learn: 0.0099166	total: 10.9s	remaining: 9.65s
5642:	learn: 0.0099127	total: 10.9s	remaining: 9.65s
5643:	learn: 0.0099124	total: 10.9s	remaining: 9.65s
5644:	learn: 0.0099077	total: 10.9s	remaining: 9.65s
5645:	learn: 0.0099034	total: 10.9s	remaining: 9.64s
5646:	learn: 0.0098984	total: 10.9s	remaining: 9.64s
5647:	learn: 0.0098916	total: 10.9s	remaining: 9.64s
5648:	learn: 0.0098884	total: 10.9s	remaining: 9.64s
5649:	learn: 0.0098825	total: 10.9s	remaining: 9.63s
5650:	learn: 0.0098766	total: 10.9s	remaining: 9.63s
5651:	learn: 0.0098737	total: 10.9s	remaining: 9.63s
5652:	learn: 0.0098700	total: 10.9s	remaining: 9.63s
5653:	learn: 0.0098615	total: 10.9s	remaining: 9.62s
5654:	learn: 0.0098563	total: 10.9s	remaining: 9.62s
5655:	learn: 0.0098520	total: 10.9s	remaining: 9.62s
5656:	learn: 0.0098454	total: 10.9s	remaining: 9.62s
5657:	learn: 0.0098390	total: 10.9s	remaining: 9.62s
5658:	lear

,Model,최적 모델의 cv_RMSE
0,GradientBoost,0.252027
1,CatBoost,0.000906


### RF

In [ ]:
def objective(trial):
  rf_param = { 'max_features':trial.suggest_int('max_features', 1, 18),
              'max_depth':trial.suggest_int('max_depth', 10, 500),
              'n_estimators':trial.suggest_int('n_estimators', 1, 100) }

  regr = RandomForestRegressor(**rf_param)
  score = cross_val_score(regr,train_final_x,train_final_y, cv=KFold(n_splits=10,shuffle=True,random_state=42), scoring="neg_mean_squared_error")
  neg_mean_squared_error_mean = score.mean()

  return neg_mean_squared_error_mean

In [ ]:
#Execute optuna and set hyperparameters
study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=500, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')

[I 2022-07-05 20:46:27,107] A new study created in memory with name: no-name-1e70bf89-3009-4318-999c-809dac195e91
[I 2022-07-05 20:46:29,240] Trial 0 finished with value: -0.20684573179760699 and parameters: {'max_features': 5, 'max_depth': 482, 'n_estimators': 100}. Best is trial 0 with value: -0.20684573179760699.
[I 2022-07-05 20:46:31,486] Trial 1 finished with value: -0.205900618823098 and parameters: {'max_features': 6, 'max_depth': 19, 'n_estimators': 99}. Best is trial 1 with value: -0.205900618823098.
[I 2022-07-05 20:46:32,715] Trial 2 finished with value: -0.2202630689032325 and parameters: {'max_features': 13, 'max_depth': 206, 'n_estimators': 37}. Best is trial 1 with value: -0.205900618823098.
[I 2022-07-05 20:46:33,096] Trial 3 finished with value: -0.236789214501057 and parameters: {'max_features': 14, 'max_depth': 67, 'n_estimators': 10}. Best is trial 1 with value: -0.205900618823098.
[I 2022-07-05 20:46:35,455] Trial 4 finished with value: -0.20658682583558288 and pa

EarlyStopping Exceeded: No new best scores on iters 30


In [ ]:
Best_trial = study.best_trial.params
optimised_rf = RandomForestRegressor(**Best_trial)
optimised_rf.fit(train_final_x,train_final_y)
mse = mean_squared_error(train_final_y, optimised_rf.predict(train_final_x))

print('최적의 하이퍼파라미터',study.best_trial.params)
print('최적 모델의 cv score', np.sqrt(mse))
print('최적 모델',study.best_trial )

new_row = {"Model": "RandomForest","최적 모델의 cv_RMSE": np.sqrt(mse)}
models = models.append(new_row,ignore_index=True)
models

최적의 하이퍼파라미터 {'max_features': 8, 'max_depth': 334, 'n_estimators': 47}
최적 모델의 cv score 0.17672174949407554
최적 모델 FrozenTrial(number=11, values=[-0.20122997489212238], datetime_start=datetime.datetime(2022, 7, 5, 20, 46, 46, 213572), datetime_complete=datetime.datetime(2022, 7, 5, 20, 46, 47, 431724), params={'max_features': 8, 'max_depth': 334, 'n_estimators': 47}, distributions={'max_features': IntUniformDistribution(high=18, low=1, step=1), 'max_depth': IntUniformDistribution(high=500, low=10, step=1), 'n_estimators': IntUniformDistribution(high=100, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=11, state=TrialState.COMPLETE, value=None)


,Model,최적 모델의 cv_RMSE
0,GradientBoost,0.252027
1,CatBoost,0.000906
2,RandomForest,0.176722


### XGB

In [ ]:
def objective(trial):

    xgb_param = {
        'objective':'reg:squarederror',
        'verbose': -1,
        'alpha': trial.suggest_uniform('alpha', 0.0, 1.0),
        'lambda': trial.suggest_uniform('lambda', 0.0, 10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-2, 1e-1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }

    regr = XGBRegressor(**xgb_param)
    score = cross_val_score(regr,train_final_x,train_final_y, cv=KFold(n_splits=10,shuffle=True,random_state=42), scoring="neg_mean_squared_error")
    neg_mean_squared_error_mean = score.mean()

    return neg_mean_squared_error_mean

In [ ]:
#Execute optuna and set hyperparameters
study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=500, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')

[I 2022-07-05 20:47:11,364] A new study created in memory with name: no-name-408c36b6-1887-460e-99b8-7e6cdcfcd1a0
[I 2022-07-05 20:47:18,161] Trial 0 finished with value: -0.22532724718652775 and parameters: {'alpha': 0.6964840686915188, 'lambda': 4.056660311534018, 'colsample_bytree': 0.9149376404888654, 'subsample': 0.6753473945178038, 'learning_rate': 0.0162727699218837, 'n_estimators': 826, 'max_depth': 4, 'min_child_weight': 55}. Best is trial 0 with value: -0.22532724718652775.
[I 2022-07-05 20:47:20,183] Trial 1 finished with value: -0.3603983174974747 and parameters: {'alpha': 0.10223250690816033, 'lambda': 2.1178785745991457, 'colsample_bytree': 0.7290415527315354, 'subsample': 0.5295847958944273, 'learning_rate': 0.04519271221775409, 'n_estimators': 402, 'max_depth': 10, 'min_child_weight': 146}. Best is trial 0 with value: -0.22532724718652775.
[I 2022-07-05 20:47:26,766] Trial 2 finished with value: -0.3062800328280124 and parameters: {'alpha': 0.1589358284797283, 'lambda':

EarlyStopping Exceeded: No new best scores on iters 30


In [ ]:
Best_trial = study.best_trial.params
optimised_xgb = XGBRegressor(**Best_trial)
optimised_xgb.fit(train_final_x,train_final_y)
mse = mean_squared_error(train_final_y, optimised_xgb.predict(train_final_x))

print('최적의 하이퍼파라미터',study.best_trial.params)
print('최적 모델의 cv score', np.sqrt(mse))
print('최적 모델',study.best_trial )

new_row = {"Model": "XGB","최적 모델의 cv_RMSE": np.sqrt(mse)}
models = models.append(new_row,ignore_index=True)
models

[21:01:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
최적의 하이퍼파라미터 {'alpha': 0.991686499088561, 'lambda': 9.088601450238277, 'colsample_bytree': 0.7444562628375917, 'subsample': 0.5647794393672237, 'learning_rate': 0.08133217592700553, 'n_estimators': 2994, 'max_depth': 15, 'min_child_weight': 1}
최적 모델의 cv score 0.0004863507507950425
최적 모델 FrozenTrial(number=25, values=[-0.2043512482820513], datetime_start=datetime.datetime(2022, 7, 5, 20, 55, 0, 620097), datetime_complete=datetime.datetime(2022, 7, 5, 20, 55, 20, 494554), params={'alpha': 0.991686499088561, 'lambda': 9.088601450238277, 'colsample_bytree': 0.7444562628375917, 'subsample': 0.5647794393672237, 'learning_rate': 0.08133217592700553, 'n_estimators': 2994, 'max_depth': 15, 'min_child_weight': 1}, distributions={'alpha': UniformDistribution(high=1.0, low=0.0), 'lambda': UniformDistribution(high=10.0, low=0.0), 'colsample_bytree': UniformDistribution(high=

,Model,최적 모델의 cv_RMSE
0,GradientBoost,0.252027
1,CatBoost,0.000906
2,RandomForest,0.176722
3,XGB,0.000486


### EXTRATREE

In [ ]:
def objective(trial):
  ex_param = { 'max_features':trial.suggest_int('max_features', 1, 18),
              'max_depth':trial.suggest_int('max_depth', 10, 500),
              'n_estimators':trial.suggest_int('n_estimators', 1, 100) }

  regr = ExtraTreesRegressor(**ex_param,random_state=42)
  score = cross_val_score(regr,train_final_x,train_final_y, cv=KFold(n_splits=10,shuffle=True,random_state=42), scoring="neg_mean_squared_error")
  neg_mean_squared_error_mean = score.mean()

  return neg_mean_squared_error_mean

In [ ]:
#Execute optuna and set hyperparameters
study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=1000)
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')

[I 2022-07-06 00:18:09,885] A new study created in memory with name: no-name-36c6284a-4526-4113-bcf9-e48d9cbc3f7b
[I 2022-07-06 00:18:12,046] Trial 0 finished with value: -0.21478410238407428 and parameters: {'max_features': 8, 'max_depth': 279, 'n_estimators': 57}. Best is trial 0 with value: -0.21478410238407428.
[I 2022-07-06 00:18:14,834] Trial 1 finished with value: -0.21191038338215687 and parameters: {'max_features': 16, 'max_depth': 159, 'n_estimators': 56}. Best is trial 1 with value: -0.21191038338215687.
[I 2022-07-06 00:18:15,684] Trial 2 finished with value: -0.21698866539789902 and parameters: {'max_features': 15, 'max_depth': 400, 'n_estimators': 27}. Best is trial 1 with value: -0.21191038338215687.
[I 2022-07-06 00:18:16,622] Trial 3 finished with value: -0.21625705831930314 and parameters: {'max_features': 15, 'max_depth': 338, 'n_estimators': 29}. Best is trial 1 with value: -0.21191038338215687.
[I 2022-07-06 00:18:17,359] Trial 4 finished with value: -0.22929282564

In [ ]:
Best_trial = study.best_trial.params
optimised_rf = ExtraTreesRegressor(**Best_trial)
optimised_rf.fit(train_final_x,train_final_y)
mse = mean_squared_error(train_final_y, optimised_rf.predict(train_final_x))

print('최적의 하이퍼파라미터',study.best_trial.params)
print('최적 모델의 cv score', np.sqrt(mse))
print('최적 모델',study.best_trial )

최적의 하이퍼파라미터 {'max_features': 13, 'max_depth': 10, 'n_estimators': 77}
최적 모델의 cv score 0.24718306200561524
최적 모델 FrozenTrial(number=411, values=[-0.19926189260988544], datetime_start=datetime.datetime(2022, 7, 6, 0, 12, 48, 773385), datetime_complete=datetime.datetime(2022, 7, 6, 0, 12, 50, 526224), params={'max_features': 13, 'max_depth': 10, 'n_estimators': 77}, distributions={'max_features': IntUniformDistribution(high=18, low=1, step=1), 'max_depth': IntUniformDistribution(high=500, low=10, step=1), 'n_estimators': IntUniformDistribution(high=100, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=411, state=TrialState.COMPLETE, value=None)


In [ ]:
Best_trial = study.best_trial.params
optimised_rf = ExtraTreesRegressor(**Best_trial)
optimised_rf.fit(train_final_x,train_final_y)
mse = mean_squared_error(train_final_y, optimised_rf.predict(train_final_x))

print('최적의 하이퍼파라미터',study.best_trial.params)
print('최적 모델의 cv score', np.sqrt(mse))
print('최적 모델',study.best_trial )

new_row = {"Model": "ExtraTrees","최적 모델의 cv_RMSE": np.sqrt(mse)}
models = models.append(new_row,ignore_index=True)
models

최적의 하이퍼파라미터 {'max_features': 15, 'max_depth': 101, 'n_estimators': 95}
최적 모델의 cv score 1.1177159821093568e-14
최적 모델 FrozenTrial(number=30, values=[-0.20786349170263124], datetime_start=datetime.datetime(2022, 7, 5, 21, 2, 31, 47777), datetime_complete=datetime.datetime(2022, 7, 5, 21, 2, 33, 458916), params={'max_features': 15, 'max_depth': 101, 'n_estimators': 95}, distributions={'max_features': IntUniformDistribution(high=18, low=1, step=1), 'max_depth': IntUniformDistribution(high=500, low=10, step=1), 'n_estimators': IntUniformDistribution(high=100, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=30, state=TrialState.COMPLETE, value=None)


,Model,최적 모델의 cv_RMSE
0,GradientBoost,2.520267e-01
1,CatBoost,9.058199e-04
2,RandomForest,1.767217e-01
3,XGB,4.863508e-04
4,ExtraTrees,1.117716e-14


### LGBM

In [ ]:
def objective(trial):

    lgbm_param = {
        'objective': 'regression',
        'verbose': -1,
        'metric': 'mse', 
        'num_leaves': trial.suggest_int('num_leaves', 2, 1024, step=1, log=True), 
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.7, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
        'max_depth': trial.suggest_int('max_depth',3, 15),
        'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
        'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),
    }
    regr = LGBMRegressor(**lgbm_param)
    score = cross_val_score(regr,train_final_x,train_final_y, cv=KFold(n_splits=10,shuffle=True,random_state=42), scoring="neg_mean_squared_error")
    neg_mean_squared_error_mean = score.mean()

    return neg_mean_squared_error_mean

In [ ]:
#Execute optuna and set hyperparameters
study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=500, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')

[I 2022-07-05 21:04:34,244] A new study created in memory with name: no-name-5b24f97f-1a8c-42ac-8720-d4ec1197a8a5
[I 2022-07-05 21:04:34,982] Trial 0 finished with value: -1.4039986798211381 and parameters: {'num_leaves': 3, 'colsample_bytree': 0.7573838534130654, 'reg_alpha': 0.8632181395587843, 'reg_lambda': 0.6866899709764029, 'max_depth': 9, 'learning_rate': 1.1317011040758548e-07, 'n_estimators': 600, 'min_child_samples': 9, 'subsample': 0.7236579873714696}. Best is trial 0 with value: -1.4039986798211381.
[I 2022-07-05 21:04:48,089] Trial 1 finished with value: -0.20450889439919212 and parameters: {'num_leaves': 365, 'colsample_bytree': 0.9102835616372518, 'reg_alpha': 0.043089329350601835, 'reg_lambda': 0.8937554642690848, 'max_depth': 12, 'learning_rate': 0.004536994407041464, 'n_estimators': 1035, 'min_child_samples': 10, 'subsample': 0.7982089163890633}. Best is trial 1 with value: -0.20450889439919212.
[I 2022-07-05 21:04:54,232] Trial 2 finished with value: -1.3980259705765

EarlyStopping Exceeded: No new best scores on iters 30


In [ ]:
Best_trial = study.best_trial.params
optimised_lgbm = LGBMRegressor(**Best_trial)
optimised_lgbm.fit(train_final_x,train_final_y)
mse = mean_squared_error(train_final_y, optimised_lgbm.predict(train_final_x))

print('최적의 하이퍼파라미터',study.best_trial.params)
print('최적 모델의 cv score', np.sqrt(mse))
print('최적 모델',study.best_trial )

new_row = {"Model": "lightgbm","최적 모델의 cv_RMSE": np.sqrt(mse)}
models = models.append(new_row,ignore_index=True)
models

최적의 하이퍼파라미터 {'num_leaves': 365, 'colsample_bytree': 0.9102835616372518, 'reg_alpha': 0.043089329350601835, 'reg_lambda': 0.8937554642690848, 'max_depth': 12, 'learning_rate': 0.004536994407041464, 'n_estimators': 1035, 'min_child_samples': 10, 'subsample': 0.7982089163890633}
최적 모델의 cv score 0.21487074777654303
최적 모델 FrozenTrial(number=1, values=[-0.20450889439919212], datetime_start=datetime.datetime(2022, 7, 5, 21, 4, 34, 984790), datetime_complete=datetime.datetime(2022, 7, 5, 21, 4, 48, 88621), params={'num_leaves': 365, 'colsample_bytree': 0.9102835616372518, 'reg_alpha': 0.043089329350601835, 'reg_lambda': 0.8937554642690848, 'max_depth': 12, 'learning_rate': 0.004536994407041464, 'n_estimators': 1035, 'min_child_samples': 10, 'subsample': 0.7982089163890633}, distributions={'num_leaves': IntLogUniformDistribution(high=1024, low=2, step=1), 'colsample_bytree': UniformDistribution(high=1.0, low=0.7), 'reg_alpha': UniformDistribution(high=1.0, low=0.0), 'reg_lambda': UniformDistrib

,Model,최적 모델의 cv_RMSE
0,GradientBoost,2.520267e-01
1,CatBoost,9.058199e-04
2,RandomForest,1.767217e-01
3,XGB,4.863508e-04
4,ExtraTrees,1.117716e-14
5,lightgbm,2.148707e-01


### Ada

In [ ]:
def objective(trial):
    ada_param = { 'learning_rate': trial.suggest_loguniform("learning_rate", 1e-8, 1e-2),
              'n_estimators':trial.suggest_int('n_estimators', 1, 100) }
    regr = AdaBoostRegressor(**ada_param,random_state=42)
    score = cross_val_score(regr,train_final_x,train_final_y, cv=KFold(n_splits=10,shuffle=True,random_state=42), scoring="neg_mean_squared_error")
    neg_mean_squared_error_mean = score.mean()

    return neg_mean_squared_error_mean

In [ ]:
#Execute optuna and set hyperparameters
study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective,n_trials=500, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')

[I 2022-07-05 21:20:34,686] A new study created in memory with name: no-name-dc5e5267-8a5f-469b-976b-d256f21d927c
[I 2022-07-05 21:20:37,164] Trial 0 finished with value: -0.33682471612503573 and parameters: {'learning_rate': 0.00010526694933594994, 'n_estimators': 49}. Best is trial 0 with value: -0.33682471612503573.
[I 2022-07-05 21:20:38,907] Trial 1 finished with value: -0.3379206454987498 and parameters: {'learning_rate': 8.072160907436542e-07, 'n_estimators': 36}. Best is trial 0 with value: -0.33682471612503573.
[I 2022-07-05 21:20:40,297] Trial 2 finished with value: -0.3373447961303036 and parameters: {'learning_rate': 3.0975844820821443e-07, 'n_estimators': 52}. Best is trial 0 with value: -0.33682471612503573.
[I 2022-07-05 21:20:42,293] Trial 3 finished with value: -0.3364371781216816 and parameters: {'learning_rate': 5.441069952656093e-08, 'n_estimators': 74}. Best is trial 3 with value: -0.3364371781216816.
[I 2022-07-05 21:20:42,394] Trial 4 finished with value: -0.3850

EarlyStopping Exceeded: No new best scores on iters 30


In [ ]:
Best_trial = study.best_trial.params
optimised_ada = AdaBoostRegressor(**Best_trial)
optimised_ada.fit(train_final_x,train_final_y)
mse = mean_squared_error(train_final_y, optimised_ada.predict(train_final_x))

print('최적의 하이퍼파라미터',study.best_trial.params)
print('최적 모델의 cv score', np.sqrt(mse))
print('최적 모델',study.best_trial )

new_row = {"Model": "Adaboost","최적 모델의 cv_RMSE": np.sqrt(mse)}
models = models.append(new_row,ignore_index=True)
models

최적의 하이퍼파라미터 {'learning_rate': 0.0009024784370187951, 'n_estimators': 37}
최적 모델의 cv score 0.542405374055149
최적 모델 FrozenTrial(number=17, values=[-0.33446860909399495], datetime_start=datetime.datetime(2022, 7, 5, 21, 20, 59, 546786), datetime_complete=datetime.datetime(2022, 7, 5, 21, 21, 0, 558224), params={'learning_rate': 0.0009024784370187951, 'n_estimators': 37}, distributions={'learning_rate': LogUniformDistribution(high=0.01, low=1e-08), 'n_estimators': IntUniformDistribution(high=100, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=17, state=TrialState.COMPLETE, value=None)


,Model,최적 모델의 cv_RMSE
0,GradientBoost,2.520267e-01
1,CatBoost,9.058199e-04
2,RandomForest,1.767217e-01
3,XGB,4.863508e-04
4,ExtraTrees,1.117716e-14
5,lightgbm,2.148707e-01
6,Adaboost,5.424054e-01


### ELA

In [ ]:
def objective(trial):
    ela_param = { 'max_iter':trial.suggest_int('max_iter', 1, 1000),
              'alpha':trial.suggest_uniform('alpha', 0.001, 1),
              'l1_ratio':trial.suggest_uniform('alpha', 0.001, 1) }
    regr = ElasticNet(**ela_param,random_state=42)
    score = cross_val_score(regr,train_final_x,train_final_y, cv=KFold(n_splits=10,shuffle=True,random_state=42), scoring="neg_mean_squared_error")
    neg_mean_squared_error_mean = score.mean()

    return neg_mean_squared_error_mean

In [ ]:
#Execute optuna and set hyperparameters
study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=500, callbacks=[early_stopping_opt])
except EarlyStoppingExceeded:
    print(f'EarlyStopping Exceeded: No new best scores on iters {OPTUNA_EARLY_STOPING}')

[I 2022-07-05 21:25:37,176] A new study created in memory with name: no-name-e28c44a8-a80f-44ff-ab52-cae7609fbd30
[I 2022-07-05 21:25:37,248] Trial 0 finished with value: -0.8400635072302933 and parameters: {'max_iter': 691, 'alpha': 0.6902560108328335}. Best is trial 0 with value: -0.8400635072302933.
[I 2022-07-05 21:25:37,339] Trial 1 finished with value: -0.7330752088398028 and parameters: {'max_iter': 338, 'alpha': 0.5775612357499215}. Best is trial 1 with value: -0.7330752088398028.
[I 2022-07-05 21:25:37,421] Trial 2 finished with value: -0.6766534920576373 and parameters: {'max_iter': 680, 'alpha': 0.504296200198177}. Best is trial 2 with value: -0.6766534920576373.
[I 2022-07-05 21:25:37,492] Trial 3 finished with value: -0.3821279364126852 and parameters: {'max_iter': 613, 'alpha': 0.12414246594847615}. Best is trial 3 with value: -0.3821279364126852.
[I 2022-07-05 21:25:37,562] Trial 4 finished with value: -1.029704619508101 and parameters: {'max_iter': 847, 'alpha': 0.82015

EarlyStopping Exceeded: No new best scores on iters 30


In [ ]:
Best_trial = study.best_trial.params
optimised_ela = ElasticNet(**Best_trial)
optimised_ela.fit(train_final_x,train_final_y)
mse = mean_squared_error(train_final_y, optimised_ela.predict(train_final_x))

print('최적의 하이퍼파라미터',study.best_trial.params)
print('최적 모델의 cv score', np.sqrt(mse))
print('최적 모델',study.best_trial )

new_row = {"Model": "ElasticNet","최적 모델의 cv_RMSE": np.sqrt(mse)}
models = models.append(new_row,ignore_index=True)
models

최적의 하이퍼파라미터 {'max_iter': 989, 'alpha': 0.006310455718356221}
최적 모델의 cv score 0.5751554607415694
최적 모델 FrozenTrial(number=24, values=[-0.3428665985139614], datetime_start=datetime.datetime(2022, 7, 5, 21, 25, 39, 86472), datetime_complete=datetime.datetime(2022, 7, 5, 21, 25, 39, 168851), params={'max_iter': 989, 'alpha': 0.006310455718356221}, distributions={'max_iter': IntUniformDistribution(high=1000, low=1, step=1), 'alpha': UniformDistribution(high=1.0, low=0.001)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=24, state=TrialState.COMPLETE, value=None)


,Model,최적 모델의 cv_RMSE
0,GradientBoost,2.520267e-01
1,CatBoost,9.058199e-04
2,RandomForest,1.767217e-01
3,XGB,4.863508e-04
4,ExtraTrees,1.117716e-14
5,lightgbm,2.148707e-01
6,Adaboost,5.424054e-01
7,ElasticNet,5.751555e-01


### Stacking

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    #모델 복제 후 fitting 시킴
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1) 
        
#Validation function
n_folds = 10

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(train_final_x)
    rmse= np.sqrt(-cross_val_score(model, train_final_x, train_final_y, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)        

In [ ]:
gb_pr = {'max_depth': 11, 'learning_rate': 0.009794933198227902, 'min_samples_leaf': 23, 'n_estimators': 976, 'loss': 'ls'}
cb_pr = {'iterations': 10637, 'od_wait': 1789, 'learning_rate': 0.09404895730815328, 'reg_lambda': 51.08340432646926, 'subsample': 0.4808467400433857, 'random_strength': 21.485995983978867, 'depth': 7, 'min_data_in_leaf': 1, 'leaf_estimation_iterations': 2, 'bagging_temperature': 0.7489081189661576, 'colsample_bylevel': 0.4720188004073166}
rf_pr = {'max_features': 8, 'max_depth': 334, 'n_estimators': 47}
xgb_pr = {'alpha': 0.991686499088561, 'lambda': 9.088601450238277, 'colsample_bytree': 0.7444562628375917, 'subsample': 0.5647794393672237, 'learning_rate': 0.08133217592700553, 'n_estimators': 2994, 'max_depth': 15, 'min_child_weight': 1}
extra_pr = {'max_features': 15, 'max_depth': 101, 'n_estimators': 95}
extra_pr_best ={'max_features': 13, 'max_depth': 10, 'n_estimators': 77}
lgbm_pr = {'num_leaves': 365, 'colsample_bytree': 0.9102835616372518, 'reg_alpha': 0.043089329350601835, 'reg_lambda': 0.8937554642690848, 'max_depth': 12, 'learning_rate': 0.004536994407041464, 'n_estimators': 1035, 'min_child_samples': 10, 'subsample': 0.7982089163890633}
ada_pr = {'learning_rate': 0.0009024784370187951, 'n_estimators': 37}
ela_pr = {'max_iter': 989, 'alpha': 0.006310455718356221}

optimised_gb = ensemble.GradientBoostingRegressor(**gb_pr)
optimised_cb = cb.CatBoostRegressor(**cb_pr)
optimised_rf = RandomForestRegressor(**rf_pr)
optimised_xgb = XGBRegressor(**xgb_pr)
optimised_lgbm = LGBMRegressor(**lgbm_pr)
optimised_extra = ExtraTreesRegressor(**extra_pr)
optimised_extra_bs = ExtraTreesRegressor(**extra_pr_best)
optimised_ada = AdaBoostRegressor(**ada_pr)
optimised_ela = ElasticNet(**ela_pr)

In [ ]:
averaged_models = AveragingModels(models = (optimised_cb,optimised_rf))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

averaged_models.fit(train_final_x,train_final_y)
test_pred= averaged_models.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5637:	learn: 0.0099552	total: 12.1s	remaining: 10.7s
5638:	learn: 0.0099474	total: 12.1s	remaining: 10.7s
5639:	learn: 0.0099382	total: 12.1s	remaining: 10.7s
5640:	learn: 0.0099340	total: 12.1s	remaining: 10.7s
5641:	learn: 0.0099324	total: 12.1s	remaining: 10.7s
5642:	learn: 0.0099249	total: 12.1s	remaining: 10.7s
5643:	learn: 0.0099163	total: 12.1s	remaining: 10.7s
5644:	learn: 0.0099096	total: 12.1s	remaining: 10.7s
5645:	learn: 0.0099030	total: 12.1s	remaining: 10.7s
5646:	learn: 0.0098983	total: 12.1s	remaining: 10.7s
5647:	learn: 0.0098924	total: 12.1s	remaining: 10.7s
5648:	learn: 0.0098891	total: 12.1s	remaining: 10.7s
5649:	learn: 0.0098827	total: 12.1s	remaining: 10.7s
5650:	learn: 0.0098756	total: 12.1s	remaining: 10.7s
5651:	learn: 0.0098706	total: 12.1s	remaining: 10.7s
5652:	learn: 0.0098656	total: 12.1s	remaining: 10.7s
5653:	learn: 0.0098548	total: 12.1s	remaining: 10.7s
5654:	learn: 0.0098484	total: 12.1s	remaining: 10.7s
5655:	lear

In [ ]:
evaluation(test_final_y1,test_pred1)

2938.507927405196

In [ ]:
averaged_models = AveragingModels(models = (optimised_cb,optimised_xgb))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

averaged_models.fit(train_final_x,train_final_y)
test_pred= averaged_models.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5638:	learn: 0.0099474	total: 12.1s	remaining: 10.7s
5639:	learn: 0.0099382	total: 12.1s	remaining: 10.7s
5640:	learn: 0.0099340	total: 12.1s	remaining: 10.7s
5641:	learn: 0.0099324	total: 12.1s	remaining: 10.7s
5642:	learn: 0.0099249	total: 12.1s	remaining: 10.7s
5643:	learn: 0.0099163	total: 12.1s	remaining: 10.7s
5644:	learn: 0.0099096	total: 12.1s	remaining: 10.7s
5645:	learn: 0.0099030	total: 12.1s	remaining: 10.7s
5646:	learn: 0.0098983	total: 12.1s	remaining: 10.7s
5647:	learn: 0.0098924	total: 12.1s	remaining: 10.7s
5648:	learn: 0.0098891	total: 12.1s	remaining: 10.7s
5649:	learn: 0.0098827	total: 12.1s	remaining: 10.7s
5650:	learn: 0.0098756	total: 12.1s	remaining: 10.7s
5651:	learn: 0.0098706	total: 12.1s	remaining: 10.7s
5652:	learn: 0.0098656	total: 12.1s	remaining: 10.7s
5653:	learn: 0.0098548	total: 12.1s	remaining: 10.7s
5654:	learn: 0.0098484	total: 12.1s	remaining: 10.7s
5655:	learn: 0.0098443	total: 12.1s	remaining: 10.7s
5656:	lear

In [ ]:
evaluation(test_final_y1,test_pred1)

3052.1932268809305

In [ ]:
averaged_models = AveragingModels(models = (optimised_extra,optimised_xgb))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

averaged_models.fit(train_final_x,train_final_y)
test_pred= averaged_models.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1

[23:43:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:43:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:44:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:44:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:44:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
evaluation(test_final_y1,test_pred1)

2892.353386281148

In [ ]:
averaged_models = AveragingModels(models = (optimised_extra,optimised_xgb,optimised_rf))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

averaged_models.fit(train_final_x,train_final_y)
test_pred= averaged_models.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1

[23:45:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:45:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:45:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:45:12] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:45:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:45:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:45:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[23:45:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [ ]:
evaluation(test_final_y1,test_pred1)

2822.3385328048075

In [ ]:
averaged_models = AveragingModels(models = (optimised_cb,optimised_xgb,optimised_extra))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

averaged_models.fit(train_final_x,train_final_y)
test_pred= averaged_models.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5638:	learn: 0.0099474	total: 12.3s	remaining: 10.9s
5639:	learn: 0.0099382	total: 12.3s	remaining: 10.9s
5640:	learn: 0.0099340	total: 12.3s	remaining: 10.9s
5641:	learn: 0.0099324	total: 12.3s	remaining: 10.9s
5642:	learn: 0.0099249	total: 12.3s	remaining: 10.9s
5643:	learn: 0.0099163	total: 12.3s	remaining: 10.9s
5644:	learn: 0.0099096	total: 12.3s	remaining: 10.9s
5645:	learn: 0.0099030	total: 12.3s	remaining: 10.9s
5646:	learn: 0.0098983	total: 12.3s	remaining: 10.9s
5647:	learn: 0.0098924	total: 12.3s	remaining: 10.9s
5648:	learn: 0.0098891	total: 12.3s	remaining: 10.9s
5649:	learn: 0.0098827	total: 12.3s	remaining: 10.9s
5650:	learn: 0.0098756	total: 12.3s	remaining: 10.9s
5651:	learn: 0.0098706	total: 12.3s	remaining: 10.9s
5652:	learn: 0.0098656	total: 12.3s	remaining: 10.9s
5653:	learn: 0.0098548	total: 12.3s	remaining: 10.9s
5654:	learn: 0.0098484	total: 12.3s	remaining: 10.9s
5655:	learn: 0.0098443	total: 12.3s	remaining: 10.9s
5656:	lear

In [ ]:
evaluation(test_final_y1,test_pred1)

2901.5671155779205

### 단일모델

In [ ]:
optimised_extra.fit(train_final_x,train_final_y)
test_pred= optimised_extra.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1
evaluation(test_final_y1,test_pred1)

2760.1072402410923

In [ ]:
optimised_xgb.fit(train_final_x,train_final_y)
test_pred= optimised_xgb.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1
evaluation(test_final_y1,test_pred1)

[23:53:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


3118.5959648265225

In [ ]:
optimised_rf.fit(train_final_x,train_final_y)
test_pred= optimised_rf.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1
evaluation(test_final_y1,test_pred1)

2828.8284523590455

In [ ]:
optimised_ela.fit(train_final_x,train_final_y)
test_pred= optimised_ela.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1
evaluation(test_final_y1,test_pred1)

43160.476111005104

In [ ]:
optimised_cb.fit(train_final_x,train_final_y)
test_pred= optimised_cb.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1
evaluation(test_final_y1,test_pred1)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
5637:	learn: 0.0099552	total: 23.3s	remaining: 20.6s
5638:	learn: 0.0099474	total: 23.3s	remaining: 20.6s
5639:	learn: 0.0099382	total: 23.3s	remaining: 20.6s
5640:	learn: 0.0099340	total: 23.3s	remaining: 20.6s
5641:	learn: 0.0099324	total: 23.3s	remaining: 20.6s
5642:	learn: 0.0099249	total: 23.3s	remaining: 20.6s
5643:	learn: 0.0099163	total: 23.3s	remaining: 20.6s
5644:	learn: 0.0099096	total: 23.3s	remaining: 20.6s
5645:	learn: 0.0099030	total: 23.3s	remaining: 20.6s
5646:	learn: 0.0098983	total: 23.3s	remaining: 20.6s
5647:	learn: 0.0098924	total: 23.3s	remaining: 20.6s
5648:	learn: 0.0098891	total: 23.3s	remaining: 20.6s
5649:	learn: 0.0098827	total: 23.3s	remaining: 20.6s
5650:	learn: 0.0098756	total: 23.3s	remaining: 20.6s
5651:	learn: 0.0098706	total: 23.3s	remaining: 20.6s
5652:	learn: 0.0098656	total: 23.3s	remaining: 20.6s
5653:	learn: 0.0098548	total: 23.3s	remaining: 20.6s
5654:	learn: 0.0098484	total: 23.3s	remaining: 20.6s
5655:	lear

3067.460442915409

In [ ]:
optimised_gb.fit(train_final_x,train_final_y)
test_pred= optimised_gb.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1
evaluation(test_final_y1,test_pred1)

2970.3059033462323

In [ ]:
optimised_ada.fit(train_final_x,train_final_y)
test_pred= optimised_ada.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1
evaluation(test_final_y1,test_pred1)

3370.675050012385

In [ ]:
optimised_lgbm.fit(train_final_x,train_final_y)
test_pred= optimised_lgbm.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1
evaluation(test_final_y1,test_pred1)

2913.5009867201384

In [ ]:
optimised_extra_bs.fit(train_final_x,train_final_y)
test_pred= optimised_extra_bs.predict(test_final_x)

test_final_y1=np.exp(test_final_y)-1
test_pred1=np.exp(test_pred)-1
evaluation(test_final_y1,test_pred1)

2699.1675496358716

# 예측

In [ ]:
test_final_x.columns

Index(['Humidity_avg', 'Get_off_subway', 'Fine_dust_concentration',
       'Temperature', 'Pub_holiday', 'Weekend', 'Vacation',
       'Precipitation_accum', 'Day_Fri', 'Day_Mon', 'Day_Sat', 'Day_Sun',
       'Day_Thu', 'Day_Tue', 'Day_Wed', 'Season_Spring', 'Season_Summer',
       'Season_Winter', 'Season_fall'],
      dtype='object')

In [ ]:
날짜 : 2022-07-09
요일 : 토
계절 : 여름
공휴일여부 : 0
쉬는날여부 : 0
방학여부 : 1
강수여부 : 0
평균 습도 : 83.54
대공원역 하차자 수 : 4237 (3개년 7월 평균)
미세먼지 :  30 (좋음의 최고치)
최고기온 : 32

In [ ]:
tomorrow_data = pd.DataFrame({'Humidity_avg' :[83.54], 'Get_off_subway':[4237], 'Fine_dust_concentration':[30],
       'Temperature':[32], 'Pub_holiday':[0], 'Weekend':[0], 'Vacation':[1],
       'Precipitation_accum':[0], 'Day_Fri':[0], 'Day_Mon':[0], 'Day_Sat':[1], 'Day_Sun':[0],
       'Day_Thu':[0], 'Day_Tue':[0], 'Day_Wed':[0], 'Season_Spring':[0], 'Season_Summer':[1],
       'Season_Winter':[0], 'Season_fall':[0]})

In [ ]:
final_pred = optimised_extra_bs.predict(tomorrow_data)
np.exp(final_pred)-1

array([6459.31822485])